In [2]:
import json
from pathlib import Path
import pandas as pd
import os

In [3]:
data_dir = '../data'

orders=pd.read_csv(data_dir+"/train_orders.csv")
ancestors=pd.read_csv(data_dir+"/train_ancestors.csv")
sample_submission=pd.read_csv(data_dir+"/sample_submission.csv")

orders_map=dict(orders[["id", "cell_order"]].values)

In [4]:
train_data={}

for filename in os.listdir(data_dir+"/train")[:1000]:
    with open(data_dir+"/train/"+filename, "r") as f:
        train_data[filename.split(".")[0]]=json.loads(f.read())

validation_data={}
for filename in os.listdir(data_dir+"/train")[1000:1200]:
    with open(data_dir+"/train/"+filename, "r") as f:
        validation_data[filename.split(".")[0]]=json.loads(f.read())

In [5]:
test_data={}

for filename in os.listdir(data_dir+"/test"):
    with open(data_dir+"/test/"+filename, "r") as f:
        test_data[filename.split(".")[0]]=json.loads(f.read())

In [6]:
train=[]
for key, notebook in train_data.items():
    notebook_len=len(orders_map[key])
    for pos, cell_id in enumerate(orders_map[key].split()):
        train.append({"notebook_id": key, "notebook_length": notebook_len, "cell_id": cell_id, "cell_position": pos, "source": train_data[key]["source"][cell_id], "cell_type": train_data[key]["cell_type"][cell_id]})
train_df=pd.DataFrame(train)

In [7]:
from collections import Counter
from operator import itemgetter

counts=Counter(" ".join(train_df["source"].values).split())

In [8]:
for w, c in list(reversed(sorted(counts.items(), key=itemgetter(1))))[:10]:
    train_df["c_"+w]=train_df["source"].apply(lambda s: 1 if w in set(s.split()) else 0)

In [13]:
train_df["relative_pos"]=train_df["cell_position"]/train_df["notebook_length"]

In [15]:
train_df

,notebook_id,notebook_length,cell_id,cell_position,source,cell_type,c_=,c_the,c_#,c_of,c_in,c_import,c_to,c_for,c_and,c_from,relative_pos
0,3348e77d273b6a,260,5c3f2239,0,# APTOS 2019 Blindness Detection\n\n### Datase...,markdown,0,1,1,1,1,0,1,1,1,1,0.000000
1,3348e77d273b6a,260,f94e96ac,1,# This Python 3 environment comes with many he...,code,0,1,1,0,1,1,1,1,0,0,0.003846
2,3348e77d273b6a,260,7fc86fbd,2,import os\nimport cv2\nimport random\nimport w...,code,0,0,0,0,0,1,0,0,0,1,0.007692
3,3348e77d273b6a,260,88f4b7f1,3,train = pd.read_csv('../input/aptos2019-blindn...,code,1,0,0,0,0,0,0,0,0,0,0.011538
4,3348e77d273b6a,260,dc5295d0,4,train.head(),code,0,0,0,0,0,0,0,0,0,0,0.015385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44178,516bba488afaa9,413,8fc8f8d9,41,#Grid Seach for XGboast\nparams = {\n '...,code,1,0,0,0,0,0,0,1,0,0,0.099274
44179,516bba488afaa9,413,7123c6c1,42,"print(""Best score is: "",gd_sr.best_score_)",code,0,0,0,0,0,0,0,0,0,0,0.101695
44180,516bba488afaa9,413,19374cbd,43,## 6. DEPLOYMENT\nThe deployment phase require...,markdown,0,1,0,1,0,0,1,1,1,0,0.104116
44181,516bba488afaa9,413,7038c6a2,44,"# If you saved model, you can use Pickle file....",code,0,0,1,0,0,0,0,0,0,0,0.106538


In [16]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(hidden_layer_sizes=(12), random_state=1, max_iter=500)
clf.fit(train_df[list(filter(lambda x: x.startswith("c_"), train_df.columns))], train_df["relative_pos"])

MLPRegressor(hidden_layer_sizes=12, max_iter=500, random_state=1)

In [27]:
test=[]

for key, notebook in test_data.items():
    notebook_len=len(test_data[key]["source"].keys())
    for cell_id in test_data[key]["source"].keys():
        test.append({"notebook_id": key, "notebook_length": notebook_len, "cell_id": cell_id, "source": test_data[key]["source"][cell_id], "cell_type": test_data[key]["cell_type"][cell_id]})
test_df=pd.DataFrame(test)
for w, c in list(reversed(sorted(counts.items(), key=itemgetter(1))))[:10]:
    test_df["c_"+w]=test_df["source"].apply(lambda s: 1 if w in set(s.split()) else 0)

test_df["predicted_position"]=clf.predict(test_df[list(filter(lambda x: x.startswith("c_"), train_df.columns))])

submit=test_df.sort_values(["notebook_id", 'predicted_position'])[["notebook_id", "cell_id"]].groupby("notebook_id").agg(lambda x: " ".join(x))
submit.columns=["cell_order"]
submit["id"]=submit.index


,cell_order,id
notebook_id,,
0009d135ece78d,8cb8d28a ddfd239c 39e937ec 7f388a41 e25aa9bd b...,0009d135ece78d
0010483c12ba9b,7844d5f8 4703bb6d 54c7cab3 02a0be6d fe66203e 4...,0010483c12ba9b
0010a919d60e4f,bac960d3 bd8fbd76 80433cf3 35cd0771 3f4a105f 2...,0010a919d60e4f
0028856e09c5b7,d22526d1 3ae7ece3 012c9d02 eb293dfc,0028856e09c5b7


In [26]:
submit

,cell_id
notebook_id,
0009d135ece78d,8cb8d28a ddfd239c 39e937ec 7f388a41 e25aa9bd b...
0010483c12ba9b,7844d5f8 4703bb6d 54c7cab3 02a0be6d fe66203e 4...
0010a919d60e4f,bac960d3 bd8fbd76 80433cf3 35cd0771 3f4a105f 2...
0028856e09c5b7,d22526d1 3ae7ece3 012c9d02 eb293dfc
